# **Finding Similar Items: Textually Similar Documents**

I am implementing the stages of finding textually similar documents based on Jaccard similarity using the shingling, minhashing, and locality-sensitive hashing (LSH) techniques and corresponding algorithms.

### Load data to google colab via google drive


In [ ]:
#http://mlg.ucd.ie/datasets/bbc.html
try:
  import os
  import pandas as pd
  import numpy as np
  import math
  import binascii
  import random
  import sympy
except:
  print("Import Error")

#Importing data
#With google docs
from google.colab import drive
drive.mount('/content/drive')
paths =['drive/My Drive/Colab Notebooks/data/bbc-fulltext/bbc/']
#OTHERWISE
'''
define paths = [HERE]
'''

def flatmap(array):
  return [element for element in innerArray for innerArray in array]

#Simple helper function
def extract_to_txt(path):
  with open(path) as f:
    contents = f.readlines()
  return contents


df = []

#Create DF with categories
for folder in paths:
  for topic in os.listdir(folder):
    topic_pth = f"{folder}{topic}/"
    for txt in os.listdir(topic_pth):
        text = extract_to_txt(topic_pth+txt)
        header = "".join(text[0])
        df.append([header,"".join(text[1::]), topic])



Mounted at /content/drive


In [ ]:
df = pd.DataFrame(df, columns=["headline", "text","label"])

print(df.columns)

type(df.iloc[0].text)

arr = []
for id, ele in df.iterrows():
  df.iloc[id].text = ele.text.replace("\n","").replace('"',"").lower()
  df.iloc[id].headline = ele.headline.replace("\n","").lower()


print(df.head(5))

Index(['headline', 'text', 'label'], dtype='object')
                               headline  \
0  iraqi voters turn to economic issues   
1     bank holds interest rate at 4.75%   
2     barclays profits hit record level   
3       deadline nears for fiat-gm deal   
4      market unfazed by aurora setback   

                                                text     label  
0  beyond the desperate security situation in ira...  business  
1  the bank of england has left interest rates on...  business  
2  barclays, the uk's third-biggest bank, has see...  business  
3  fiat and general motors (gm) have until midnig...  business  
4  as the aurora limped back to its dock on 20 ja...  business  



### **1. Shingling function**

Shingling constructs k–shingles of a given length k (e.g., 10) from a given document, computes a hash value for each unique shingle and represents the document in the form of an ordered set of its hashed k-shingles.

In [ ]:
def shingling(k=5,txt=""):
  arr = []
  for i in range(0,len(txt)-k):
    arr.append(txt[i:i+k])
  return set(arr)

def toHash(shing_list):
  return [binascii.crc32(bytes(x.encode('utf-8'))) for x in shing_list]

shinglesDoc1 = shingling(8,df.iloc[0].text)
shinglesDoc2 = shingling(8,df.iloc[1].text)
print(shinglesDoc1)

{'scientis', 'er, who ', 'vide; th', 'c bankin', 'ttracted', 'think th', 'f job cr', 'ol unive', 'reation,', 'iorities', 'n made w', 'he jorda', 'urity an', 'ckwatch ', ' says in', 'e of eco', 'index. b', 'rdinary ', 't months', 's still ', 'ion says', 'h the da', 'ut of wo', 'ment rat', 'l as boo', 'nemploym', 't bank e', 'ity, the', ' jordani', 'rship of', 'ty has b', 'rces can', 'about wh', 'al baath', 'ry, but ', 'rs and g', 'hat look', ' failed ', 'ing to w', 'about he', 'en made,', ' as boos', 'that are', 'o allow ', 'g into t', 'contract', 'reign ow', 'ry contr', 'nd beyon', "'s infra", 'tate tak', 'g role i', 'nment wi', 'es. obse', 'sity pol', 'joyed a ', 'te takin', 's soon a', 'rding to', 't presen', 'we estim', 's remain', 'ccustome', 'electric', 'n terms ', 'nce wher', 'things, ', ' get poo', 'ing dire', 'ness peo', ' adverts', 'firms in', 'line whi', "'s black", 'ices and', 'ls at th', 't across', 'have bee', 'the grow', 'al secur', 't & fina', 'by many ', 'es mr ha', ' i

### **2. CompareSets function**
CompareSets computes the Jaccard similarity of two sets of integers – two sets of hashed shingles.

#### 2.1 For two Sets
This is a efficient implementation for comapring two sets of shingles.

In [ ]:
def CompareSets(set1, set2):
    list1 = list(set1)
    list2 = list(set2)
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    similarity = float(intersection) / union
    return similarity

# Example
similarity = CompareSets(shinglesDoc1, shinglesDoc2)

In [ ]:
%%time
print(CompareSets(shinglesDoc1, shinglesDoc2))

0.015592515592515593
CPU times: user 3.31 ms, sys: 0 ns, total: 3.31 ms
Wall time: 3.3 ms


#### 2.2 With Characteristic Matrix

With increased docs, the matrix becomes more sparse. Resulting in lower performances.

1. Create Charactersitc Matrix
2. Apply JacSimilarity on two columns (2 docs) **Text fett markieren**

In [ ]:

"""
Visualize a collection of sets as a characteristic matrix
Rows correspond to elements (shingles) of the universal set.
Columns correspond to sets (documents)
p. 24
"""

def createCharacteristicMatrix(docs):
  sMatrix = pd.DataFrame([],columns=["shingle"])
  for idx, i in enumerate(docs):
    sh = shingling(txt=i)
    hsh = pd.DataFrame(toHash(sh),columns=["shingle"])
    bol_vals = pd.DataFrame([1 for idx in range(len(hsh))], columns=["s"+str(idx)])
    sRow = pd.concat((hsh, bol_vals), axis=1)
    sMatrix = pd.merge(sMatrix, sRow, on='shingle', how='outer').fillna(0)
  return sMatrix

"""
1 1 a 1 in both columns
1 0 b columns are different
0 1 c
0 0 d 0 in both columns
Denote,
A = the number of rows of type a;
B = the number of rows of type b, etc.
Sim (C1, C2) = A /(A +B +C).
p.26
"""

# row 0 is the shingle
def calculateSimJac(charMatrix,docCol1=1, docCol2=2, p=True):
  a=0; b=0; c=0;
  for i in charMatrix.iterrows():
    # i[0] --> rowNumber; i[0][0] --> Shingle Number; i[0][1] --> first document
    dc0 = i[1][docCol1]; dc1 = i[1][docCol2]
    if dc0 == 1 and dc1 == 1: a+=1
    elif dc0 != dc1 and dc0 == 1 or dc1 == 1 : b+=1
    else: c += 1
  if p : print(f"A:B:C -> {a,b,c}, Sum of A+B+C -> {a+b+c}, CheckLen of Matrix -> {len(charMatrix)}")
  jacSim = a / (a+b+c*0)
  return jacSim


print(df.text[0:10])
cM = createCharacteristicMatrix(df.text[0:10])
print(cM)
jacSim = calculateSimJac(cM,1,3)
print("jacSimiliarity:", jacSim)



0    beyond the desperate security situation in ira...
1    the bank of england has left interest rates on...
2    barclays, the uk's third-biggest bank, has see...
3    fiat and general motors (gm) have until midnig...
4    as the aurora limped back to its dock on 20 ja...
5    us phone company sbc communications said it ex...
6    the european commission has officially launche...
7    japanese communications firm softbank has wide...
8    shares in deutsche boerse have risen more than...
9    uk retail sales fell in december, failing to m...
Name: text, dtype: object
          shingle   s0   s1   s2   s3   s4   s5   s6   s7   s8   s9
0      4216709056  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1       256847691  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2      3431501741  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3       782316916  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4      2201171154  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
...           ..

In [ ]:
text1 =  "this is a test to understand better what we are doing! yippie!"
text2 =  "this is a test to understand better what we are doing! yippie!"
text3 =  "this is a test to not forget  what we are doing! yippie! Same but different"


test_mat =  pd.DataFrame([text1,text2,text3], columns=["text"])
#print(test_mat.text)
cM_test_1 = createCharacteristicMatrix(test_mat.text)
print(cM_test_1)
jacSim_test_1 = calculateSimJac(cM_test_1,1,2)
jacSim_test_2 = calculateSimJac(cM_test_1,1,3)
print("jacSimiliarity:", jacSim_test_1, "and", jacSim_test_2)


       shingle   s0   s1   s2
0    366920414  1.0  1.0  0.0
1   1285067948  1.0  1.0  1.0
2   1861508059  1.0  1.0  1.0
3   3220349980  1.0  1.0  1.0
4   1689852086  1.0  1.0  0.0
..         ...  ...  ...  ...
86  4077143596  0.0  0.0  1.0
87  2794877598  0.0  0.0  1.0
88  2679835442  0.0  0.0  1.0
89   470867650  0.0  0.0  1.0
90  3070962247  0.0  0.0  1.0

[91 rows x 4 columns]
A:B:C -> (57, 0, 34), Sum of A+B+C -> 91, CheckLen of Matrix -> 91
A:B:C -> (36, 55, 0), Sum of A+B+C -> 91, CheckLen of Matrix -> 91
jacSimiliarity: 1.0 and 0.3956043956043956


### **Task3. Minhashing**
MinHashing builds a minHash signature (in the form of a vector or a set) of a given length n from a given set of integers (a set of hashed shingles).

#### 3.2 HashSignature with Permutation

In [ ]:
%%time
def createPermSignVect(index,vec):
  for e in zip(index,vec):
    if e[1] == 1: return e[0]


def signMatrixPermutations(charMatrix, permutations, createVec):
  assert permutations > 0
  signMatrix = pd.DataFrame(columns=charMatrix.columns[1:])
  for _ in range(permutations):
    #permutate all rows.
    shuffled = charMatrix.sample(frac=1)
    ll = list();
    for e in shuffled.columns[1:]:
      ll.append(createVec(shuffled.index,shuffled[e]))
    signMatrix.loc[len(signMatrix)] = ll
  return signMatrix


#createPermSignVect(cM.index,cM["s0"])
signMatPer = signMatrixPermutations(cM,100,createPermSignVect)
print(signMatPer)

      s0    s1    s2    s3    s4    s5    s6     s7     s8     s9
0   2360  3707  5960  3175  3175  3175  3175   9893   3175  11170
1     31  4734  6065  6505  7416  8631  3337  10116  10297  11244
2   2875  4800  4482  6939  8376  8426  9018   4482     34   4482
3   1750  5051  5626  2593  1558  2593  9704   9817  10480  11651
4     14  2306  5199  5199  7199  7199  9587   1585   7015   5199
..   ...   ...   ...   ...   ...   ...   ...    ...    ...    ...
95   602  4747  5739  6693  3309  6800  5639   2866   6800   3493
96  2667  3986  3589  4068  2549  4319  9582   8721   4068   4105
97  2153  4106  5910  6596  7855  3339  9543   1879  10913  11509
98  2527  3944  5898  6718  1962  6063  9417  10119   7224  11290
99  2421  4683  5990  4683  8296  4683  9543  10050  10329  11220

[100 rows x 10 columns]
CPU times: user 379 ms, sys: 2.82 ms, total: 382 ms
Wall time: 385 ms


####  MinHashing with Hashfunctions

from: https://mccormickml.com/2015/06/12/minhash-tutorial-with-python-code/ <br/>
So here’s how you compute the MinHash signature for a
given document. Generate, say, 10 random hash functions. Take the first hash function,
and apply it to all of the shingle values in a document. Find the minimum hash value produced (hey, “minimum hash”, that’s the name of the algorithm!)
and use it as the first component of the MinHash signature. Now take the second hash function, and again find the minimum resulting hash value,
and use this as the second component. And so on.


In [ ]:

'''
Take k (e.g., k = 100) independent hash functions, e.g.,
h(x) = (ax + b) % c
'''
#Create Hashfunctions
def createMinHashCoeff(amount,coeff = list()):

  # own values = a = random.randint(1,100000); b = random.randint(1,100000); c =  sympy.randprime(2,97)
  # according to: https://mccormickml.com/2015/06/12/minhash-tutorial-with-python-code/
  maxShingleID = 2**32-1
  # http://compoasso.free.fr/primelistweb/page/prime/liste_online_en.php
  nextPrime = 13570    #4294967311 next higher prime of 2**32-1
                       #13570 shingles we have in nine docuemtns

  assert amount > 0
  arr = []
  for _ in range(amount):
    #a = random.randint(1,100000); b = random.randint(1,100000); c =  sympy.randprime(2,97)
    a = random.randint(1, maxShingleID); b = random.randint(1,maxShingleID); c = nextPrime
    arr.append((a,b,c))
  cf = set(arr)

  return cf if len(cf) == amount else createMinHashCoeff(amount)



In [ ]:
'''
Take k (e.g., k = 100) independent hash functions, e.g.,
h(x) = (ax + b) % c
'''

### For a single Row
"""
What you do is hash each element of A and each element of B, and look at the minimum hash value of each set.
As it turns out, the probability of these two hashes being equal is exactly the Jaccard similarity of the two sets. Wow -- that is elegant!
http://web.eecs.utk.edu/~jplank/plank/classes/cs494/494/notes/Min-Hash/index.html
"""
def createMinHashRow(hashedShingle,col,index,coeffList):
  hcol = []
  for coeffs in coeffList:
    min_val = math.inf; min_pos = math.inf #  i have seen implementations using min_val and min_pos -> i guess its the same in the end? Yes. Scores for testing reamined the same.
    for i in zip(hashedShingle,col,index):
      if i[1] == 1:
        #(ax + b) % c
        h = (int(coeffs[0])*int(i[0]) + int(coeffs[1])) % int(coeffs[2])
        if min_val > h:
          min_val = h
          min_pos = i[2]
    #print(min_val)
    hcol.append(min_pos) # we are using the position of min_val instead of the actual min_val?
  return hcol


# Function for just a single row
'''
mHsCol = createMinHashRow(cM["shingle"].values,cM["s0"].values,cM.index, coeffs)
mHsCol1 = createMinHashRow(cM["shingle"].values,cM["s1"].values,cM.index, coeffs)
# with own randomly set coefficients
#[0,0,0,0]
#[0,0,0,0]
#
print(mHsCol)
'''

' \nmHsCol = createMinHashRow(cM["shingle"].values,cM["s0"].values,cM.index, coeffs)\nmHsCol1 = createMinHashRow(cM["shingle"].values,cM["s1"].values,cM.index, coeffs)\n# with own randomly set coefficients\n#[0,0,0,0] \n#[0,0,0,0]\n#\nprint(mHsCol) \n'

In [ ]:
%%time
"""
Thus, we can form from matrix M a signature matrix, in which
the ith column of M is replaced by the minhash signature for (the set of) the
ith column.
Note that the signature matrix has the same number of columns as M but
only n rows. Even if M is not represented explicitly, but in some compressed
form suitable for a sparse matrix (e.g., by the locations of its 1’s), it is normal
for the signature matrix to be much smaller than M
"""

# CREATE COEFFICIENTS FOR HASHFUNCTIONS
coeffs = createMinHashCoeff(100)

def signMatrix(minHashRow,charMatrix,coeffList):
  #use either : f"h(x)=({x[0]}*x + {x[1]} % {x[2]}" or h[i] as keystr
  #keystr = signMat = pd.DataFrame([f"h({i})" for i in range(1,len(coeffList)+1)], columns=["hashfunction"])
  signMat = pd.DataFrame([])
  for e in charMatrix.columns[1:]:
    charM = charMatrix
    signMat[e] = minHashRow(charM["shingle"].values,charM[e],charM.index, coeffList)
  return signMat

signMat = signMatrix(createMinHashRow, cM,coeffs)
print(signMat)

      s0    s1    s2    s3    s4    s5    s6     s7     s8     s9
0   1683  4645   289  3455  7604   289  8158   1793  10579   4645
1   2359  4143  5666  6689  4143   581  9239   4273  11073   4143
2     43   735   735    43    43    43    43     90     43     43
3    855  2047   855   925  2047   925   925    925  10654    925
4   1767  1767  5665  6150  8288  8582  9386  10091   5665   1767
..   ...   ...   ...   ...   ...   ...   ...    ...    ...    ...
95   135   987  1069   422   135  1107   987   1107    987   1107
96   627   627  2035   627   627   627   627   9730   9815   2035
97  1950  1950  1950  1950  1950  6071  9675  10068  10148   1950
98  2366  4142  5750  4881  8402  8580  9494   4895   1248   1248
99    57  4050    57  6680  8389  8582   720     18  10552  11748

[100 rows x 10 columns]
CPU times: user 5.21 s, sys: 0 ns, total: 5.21 s
Wall time: 5.24 s


#### 4. Compare Signatures
A class CompareSignatures estimates the similarity of two integer vectors – minhash signatures – as a fraction of components in which they agree.

In [ ]:
def getSigSim(a,b):
  assert len(a) == len(b)
  count = 0
  for i in zip(a,b):
     if i[0] == i[1]: count +=1
  return count / len(a)


sh1 = signMat["s0"].values; sh2 = signMat["s1"].values
sp1 = signMatPer["s0"].values; sp2 = signMatPer["s1"].values

sH = getSigSim(sh1,sh2)
sP = getSigSim(sp1,sp2)
# Estimates are often similiar.
# sh and sP are changing, depending on randomness.

print(sH, sP, "for jacSim:",calculateSimJac(cM,1,2, p=False))
#For 300 hashfunctions / permutations 0.12056737588652482 0.10909090909090909
#For 600 hashfunctions / permutations 0.14145383104125736 0.08548707753479125
#For 100 hashfunctions / permutations 0.10204081632653061 0.10309278350515463


#With test matrix
coeffs_test = createMinHashCoeff(1000)
signMatTest = signMatrix(createMinHashRow, cM_test_1,coeffs_test)
signMatPerTest = signMatrixPermutations(cM_test_1,1000,createPermSignVect)

sht1 = signMatTest["s0"].values; sht2 = signMatTest["s1"].values;  sht3 = signMatTest["s2"].values
spt1 = signMatPerTest["s0"].values; spt2 = signMatPerTest["s1"].values;  spt3 = signMatPerTest["s2"].values

sHt = getSigSim(sht1,sht2)
sPt = getSigSim(spt1,spt2)

#Checking is similar values are similar.
print(sHt, sPt, "for jacSim:",calculateSimJac(cM_test_1,1,2, p=False))
#For mostly similiar strings:
sHt1 = getSigSim(sht1,sht3)
sPt1 = getSigSim(spt1,spt3)
print("NOTE, we set a very high  hash / permutation number for test --> very accurate resutls")
print(sHt1, sPt1, "for jacSim:",calculateSimJac(cM_test_1,1,3, p=False))

0.14 0.09 for jacSim: 0.08679319889284302
1.0 1.0 for jacSim: 1.0
NOTE, we set a very high  hash / permutation number for test --> very accurate resutls
0.39 0.408 for jacSim: 0.3956043956043956


### Locality Sensitive Hashing (LSH)

(Optional task for extra 2 bonus points) A class LSH that implements the LSH technique: given a collection of minhash signatures (integer vectors) and a similarity threshold t, the LSH class (using banding and hashing) finds candidate pairs of signatures agreeing on at least a fraction t of their components.

In [ ]:
#check https://www.youtube.com/watch?v=e_SBq3s20M8&ab_channel=JamesBriggs @ 21min
def splitVec(vec,b,r):
  assert len(vec) % b == 0
  assert len(vec) / b % r == 0
  subvecs = []
  for i in range(0,len(vec)-1,r):
    subvecs.append(vec[i:i+r])
  return subvecs

#BBC - for hashMatrix
b1 = splitVec(signMat["s1"].values,20,5)
b2 = splitVec(signMat["s2"].values,20,5)

# for testdata
t1 = splitVec(signMatTest["s1"].values,20,5)
t2 = splitVec(signMatTest["s2"].values,20,5)
print(t1)

[array([32, 51, 27, 41, 43]), array([47, 31, 48, 23, 25]), array([10, 28, 10,  2, 56]), array([18, 36, 49, 11,  6]), array([ 7, 18, 51,  2, 37]), array([13, 47, 27, 49, 37]), array([32, 10, 48, 11,  2]), array([55,  2, 19, 49, 39]), array([10, 47, 29, 43, 14]), array([33, 26, 51, 42, 55]), array([44,  3, 27, 12, 25]), array([42, 53, 10, 33, 44]), array([26,  7, 54, 28, 54]), array([44, 11, 31, 18, 22]), array([38,  9, 16, 35, 28]), array([40,  7, 44, 16,  4]), array([21, 41, 26, 41,  0]), array([42, 29, 26,  5,  1]), array([23, 25,  4,  8, 20]), array([15, 22, 49, 17, 46]), array([27, 54,  3, 44,  1]), array([ 3, 10,  9, 33, 51]), array([27, 53, 51, 25, 14]), array([25, 44, 34, 22, 49]), array([45,  7, 19, 51, 26]), array([12, 29, 40,  3,  3]), array([38, 31,  0, 27,  9]), array([55, 49,  1, 14, 25]), array([ 2, 40, 52, 30, 55]), array([41, 44,  6, 17, 30]), array([11, 39, 36, 48, 13]), array([19,  3, 42,  0,  6]), array([21, 45, 34, 25, 28]), array([28, 55,  8, 55,  9]), array([32, 25

In [ ]:
def getCandidatePairs(v1,v2, t):
  assert len(v1) == len(v2)
  for e in zip(v1,v2):
    # we can reuse getSigSim / used for comparing whole signatures
    if getSigSim(e[0],e[1]) > t:
      return True
  return False

#is candidate pair?
# BBC
print("t = 0.5",getCandidatePairs(b1,b2,0.5), "| t = 0.1",getCandidatePairs(b1,b2,0.1))
# Testdata
print("t = 0.9",getCandidatePairs(t1,t2,0.99), "| t = 0.1",getCandidatePairs(t1,t2,0.1))

t = 0.5 False | t = 0.1 True
t = 0.9 True | t = 0.1 True


#### Testprogramm

In [ ]:
def findSimilarDocuments(signMatrix, t):
  similarItems = []

  for i in range(len(signMatrix.columns)):
    for j in range(i+1,len(signMatrix.columns)):
      if getSigSim(signMatrix["s"+str(i)].values, signMatrix["s"+str(j)].values) > t:
         similarItems.append({i:j})
  return similarItems

print(len(signMat.columns))
sI010 = findSimilarDocuments(signMat,0.10)
sI015 = findSimilarDocuments(signMat,0.15)
sI030 = findSimilarDocuments(signMat,0.30)
print("Similar Documents with 0.10 Similariy: ", sI010)
print("Similar Documents with 0.15 Similariy :", sI015)
print("Similar Documents with 0.30 Similariy :", sI030)

10
Similar Documents with 0.10 Similariy:  [{0: 1}, {0: 3}, {0: 4}, {0: 9}, {1: 3}, {1: 4}, {1: 9}, {3: 4}, {3: 5}, {3: 6}, {3: 9}, {4: 8}, {4: 9}, {6: 9}, {7: 9}, {8: 9}]
Similar Documents with 0.15 Similariy : [{1: 9}, {3: 9}, {4: 9}]
Similar Documents with 0.30 Similariy : []
